# ０　元ネタ：ここを参考に実装しました
 - アプリ開発：Jupyter上でDashを使えるjupyter_dash
  - https://qiita.com/OgawaHideyuki/items/725f4ffd93ffb0d30b6c

# １　必要なライブラリのインストール

In [ ]:
!pip install jupyter_dash dash-bootstrap-components
!pip3 install mecab-python3 unidic-lite
!pip3 install wordcloud
!apt-get -y install fonts-ipafont-gothic

# ２　ライブラリのインポート

## Webアプリ関係

In [ ]:
from jupyter_dash import JupyterDash 
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
from io import BytesIO
import base64

## wordcloud関係

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from wordcloud import WordCloud

## 自作ライブリ（関数）のインポート

In [ ]:
import sys
sys.path.append('//content/drive/MyDrive/0_HanPy41/1_Webアプリケーション')

import connpass_api

- **connpass_api.py**

```python
import MeCab
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import requests

URL = 'https://connpass.com/api/v1/event'

def get_words(nickname, count):
    url_query = f'{URL}/?nickname={nickname}&count={count}&order=2'
    response = requests.get(url_query).json()
    if response['results_available'] == 10000:
        return "該当ありません"
    events = response['events']

    text = ""
    for event in events:
        for key, item in event.items():
            if key == "title":
                e_title = item
            try:
                if key == "series":
                    s_title = item['title']
            except:
                s_title = ""
        text += f'{e_title} {s_title} '
    
    mecab = MeCab.Tagger()
    parts = mecab.parse(text)
    nouns = []
    for part in parts.split('\n')[:-2]:
        if '名詞' in part.split('\t')[4]:
            nouns.append(part.split('\t')[0])
            
    return ' '.join(nouns)

if __name__ == '__main__':
    main()
```



### 自作ライブラリのテスト

In [ ]:
words = connpass_api.get_words('malo21st', 20)

fpath = '/usr/share/fonts/truetype/fonts-japanese-gothic.ttf'
wc = WordCloud(width=1200, height=800, background_color='white', font_path=fpath)
wc.generate(words)
plt.figure(figsize=(12,8))
plt.axis("off")
plt.imshow(wc)


# ３　ワードクラウドＷｅｂアプリの実装



In [ ]:
fpath = '/usr/share/fonts/truetype/fonts-japanese-gothic.ttf'
COUNT = 20

app = JupyterDash(__name__,
                external_stylesheets=[dbc.themes.SUPERHERO],
                # external_stylesheets=[dbc.themes.BOOTSTRAP],
                meta_tags=[{
                    'name': 'viewport',
                    'content': 'width=device-width, initial-scale=1.0'
                    }]
                )

# View
title = html.Div([
    dbc.Row([
        dbc.Col(
            dbc.Alert(html.H3("Connpass Word Cloud"), 
                      color="primary", style={'text-align':'center'}
                      )
        ),
    ], justify="center")
])

input_name = html.Div([
    dbc.Row([
        dbc.Col(
            html.H4('nick name：'), width="auto"
        ),
        dbc.Col(
            dcc.Input(
                id = 'id_name',
                placeholder='Press Enter to apply',
                type='text',
                value='',
                debounce=True,
                autoFocus=True,
            ), width="auto"
        ),
    ], no_gutters=True, justify="center")
])

output_wc = html.Div([
    dbc.Row([
        dbc.Col(
            dcc.Loading(id="id_loading",
                children=[
                        html.Img(id="id_wc"),
                ],
                type="default",
            ), width="auto"
        ),
    ], justify="center")
], style={'margin':'10px 0px 0px 0px'}) # マージン：上　右　下　左

app.layout = dbc.Container([
                            title, 
                            input_name, 
                            output_wc,
                            ])

# Controller
@app.callback(
	Output('id_wc', 'src'),
	[Input('id_name', 'value')],
)
# Model
def make_image(nick_name):
    if nick_name == '':
        words = 'ニックネームを入力して下さい'
    else:
        words = connpass_api.get_words(nick_name, COUNT)

    wc = WordCloud(width=350, height=350, background_color='white', font_path=fpath)
    wc.generate(words)
    img = BytesIO()
    wc.to_image().save(img, format='PNG')
    return f'data:image/png;base64,{base64.b64encode(img.getvalue()).decode()}'

# ４　Ｗｅｂアプリをローカル環境で実行

In [ ]:
app.run_server(mode="inline")

# ５　Ｗｅｂアプリのデプロイ

## externalモードでWebアプリを起動

In [ ]:
app.run_server(mode='external', host="localhost", port=8050, debug=False)

## トンネリングで公開ＵＲＬを発行

In [ ]:
# cloudflaredのインストール　＆　localhostの8050ポートのトンネリングした公開URLを発行
!wget https://bin.equinox.io/c/VdrWdbjqyF/cloudflared-stable-linux-amd64.deb
!dpkg -i cloudflared-stable-linux-amd64.deb
!cloudflared tunnel --url localhost:8050